### Parameters

These are changed by Papermill in the data pipeline.

In [ ]:
DEBUG = True
HIDE_TITLE = True

# Metrics


## Setup


### Imports


In [ ]:
# # Check is flaky in notebooks
# pyright: reportUnnecessaryTypeIgnoreComment=none

from pathlib import Path
from shutil import copy
import warnings

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

from boilerdata.axes_enum import AxesEnum as A  # noqa: N814
from boilerdata.models.project import Project
from boilerdata.stages.common import per_run
from boilerdata.stages.modelfun import model_with_uncertainty
from boilerdata.stages.notebooks.common import FLOAT_SPEC  # type: ignore  # magic
from boilerdata.stages.notebooks.common import chdir_to_nearest_git_root_and_get_project
from boilerdata.stages.notebooks.metrics import add_units, plot_new_fits, tex_wrap

### Data


In [ ]:
if DEBUG:
    proj = chdir_to_nearest_git_root_and_get_project()
else:
    proj = Project.get_project()

In [ ]:
meta = [col.name for col in proj.axes.meta]
errors = proj.params.free_errors
fits = proj.params.free_params
df_in = pd.read_csv(
    proj.dirs.file_results,
    index_col=(index := [A.trial, A.run]),
    parse_dates=index,
    dtype={col.name: col.dtype for col in proj.axes.cols},
)
limits_in = {
    A.T_5: (50, 300),
    A.T_s_err: (0, 20),
    A.q_s_err: (0, 5),
    A.k_err: (0, 10),
    A.h_a_err: (0, 40),
    A.h_w_err: (0, 40),
}

### Notebook

In [ ]:
_ = display()
"""Suppress implicit outputs. Unlike `;`, doesn't get formatted out by `black`."""
_  # type: ignore

In [ ]:
%precision %$FLOAT_SPEC

### Plotting


In [ ]:
"""This warning fires unnecessarily when Seaborn or Pandas plots are placed in existing
axes. This warning can't be caught in context of `warnings.catch_warnings()` because
it fires *after* a cell finishes executing. So we have to disable this globally."""
warnings.filterwarnings(
    category=UserWarning,
    action="ignore",
    message="This figure includes Axes that are not compatible with",
)

sns.set_theme(
    context="notebook",
    style="whitegrid",
    palette="bright",
    font="sans-serif",
)

plt.style.use(style=proj.dirs.mpl_base)
if HIDE_TITLE:
    plt.style.use(style=proj.dirs.mpl_hide_title)

## Metrics


In [ ]:
Path(proj.dirs.file_pipeline_metrics).write_text(
    df_in[errors]
    .agg(["median", "max"])
    .rename(axis="index", mapper={"median": "med"})
    .to_json(),
    encoding="utf-8",
)

_  # type: ignore

## Plots


### Error and temperature

In [ ]:
cols = [A.joint, A.T_5, *errors]
df, col_to_unitcol = add_units(df_in, proj)
col_to_unitcol = {k: v for k, v in col_to_unitcol.items() if k in cols}
df = df[[col_to_unitcol[col] for col in cols]]
df, unitcol_to_texunitcol = tex_wrap(df)
c = dict(zip(col_to_unitcol, unitcol_to_texunitcol.values()))

joints = dict(
    paste=".",
    epoxy="^",
    solder="s",
    none="D",
)
limits = {c[k]: v for k, v in limits_in.items() if k in c}

for error, path in zip(
    errors,
    [
        proj.dirs.plot_error_T_s,
        proj.dirs.plot_error_q_s,
        proj.dirs.plot_error_h_a,
    ],
):
    jg = sns.JointGrid()
    jg.ax_marg_x.remove()

    common = dict(
        data=df,
        y=c[error],
    )
    sns.scatterplot(
        ax=jg.ax_joint,
        **common,
        x=c[A.T_5],
        hue=c[A.joint],
        style=c[A.joint],
        markers=joints,  # type: ignore  # seaborn
        edgecolor="gray",
        hue_order=joints.keys(),
        alpha=0.9,
    )
    sns.histplot(
        ax=jg.ax_marg_y,
        **common,
        stat="count",
        bins=16,  # type: ignore
        color="gray",
    )

    jg.ax_joint.set_xlim(limits[c[A.T_5]])
    jg.ax_joint.set_ylim(limits[c[error]])  # type: ignore
    jg.figure.savefig(path)

### New model fits


In [ ]:
if proj.params.do_plot:
    per_run(df, plot_new_fits, proj, model_with_uncertainty)
    if figs_src := sorted(proj.dirs.new_fits.iterdir()):
        figs_src = (
            figs_src[0],
            figs_src[len(figs_src) // 2],
            figs_src[-1],
        )
        figs_dst = (
            proj.dirs.plot_new_fit_0,
            proj.dirs.plot_new_fit_1,
            proj.dirs.plot_new_fit_2,
        )
        for fig_src, fig_dst in zip(figs_src, figs_dst):
            copy(fig_src, fig_dst)